In [ ]:
!pip install transformers accelerate datasets

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

# Modelli a confronto
INSTRUCT_MODEL = "google/gemma-3-1b-it"

# Carichiamo tokenizer condiviso
tokenizer = AutoTokenizer.from_pretrained(INSTRUCT_MODEL)

# Carichiamo i due modelli
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

inst_model = AutoModelForCausalLM.from_pretrained(INSTRUCT_MODEL, torch_dtype=torch.float16).to(device)


/Users/dariomazzola/Desktop/Fine-Tuning-SLM/labcamp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


# 3. How parameters affect model performance

In [2]:
def generate_with_params(model, prompt, temperature=1.0, top_p=1.0, top_k=0, do_sample=False, max_new_tokens=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
    return tokenizer.decode(output[0], skip_special_tokens=False)

prompt = "Racconta una breve storia su un gatto magico."

## 1. Greedy decoding: at each step, pick the token with the highest probability. 

In [3]:
exp = {"description": "Greedy decoding (baseline, no sampling)", "params": {"do_sample": False}}
print("="*100)
print(f"👉 {exp['description']}")
result = generate_with_params(inst_model, prompt, **exp["params"])
print(result)

The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


👉 Greedy decoding (baseline, no sampling)
<bos>Racconta una breve storia su un gatto magico.

Il cielo era di un blu intenso, quasi viola, e la pioggia cadeva a dirotto. In un piccolo villaggio, nascosto tra le montagne, viveva un gatto di nome Silas. Silas non era un gatto qualsiasi; era un gatto magico.

Ogni notte, quando la luna piena illuminava il villaggio, Silas si trasformava in un piccolo umano, indossando abiti semplici e camminando per le strade. Durante il giorno, si nascondeva tra i cespugli e le rocce, osservando il mondo con i suoi occhi gialli e penetranti.

Un giorno, un giovane ragazzo di nome Leo, che era in difficoltà, si imbatté in Silas. Leo era un artista, ma la sua creatività era stata soffocata dalla paura. Aveva perso il suo colore, la sua ispirazione, e non riusciva più a dipingere.

Silas, vedendo il dolore di Leo, si trasformò in un uomo e lo invitò a sedersi accanto a lui. Leo, sorpreso, accettò.

Silas, con un tocco delicato, gli parlò con una voce profon

## 2. Sampling

The most standard way of generating sequences is to use the distributions predicted by a model, without any modifications.

> We sample until the _eos_ token is generated.

<video width="2000" controls>
  <source src="src/generation_example.mp4" type="video/mp4">
</video>


In [4]:
exp = {"description": "Sampling semplice, temperature=1.0", "params": {"do_sample": True, "temperature": 1.0}}

print("="*100)
print(f"👉 {exp['description']}")
result = generate_with_params(inst_model, prompt, **exp["params"])
print(result)

👉 Sampling semplice, temperature=1.0
<bos>Racconta una breve storia su un gatto magico.

Il pelo di Silvanus era un lampo di blu, e i suoi occhi, scintillanti come stelle. Viveva in un piccolo villaggio nascosto tra le montagne, dove le case erano fatte di legno scuro e gli alberi sussurravano storie antiche. Silvanus non era un gatto normale. Aveva un dono: poteva trasformare l'acqua in magia.

Un giorno, un giovane uomo, Elias, si imbarcò in un viaggio per trovare una rara pietra che si diceva potesse curare una malattia. Mentre attraversava il villaggio, si imbatté in Silvanus. Elias pensò che fosse un animale selvatico, ma quando Silvanus lo guardò, i suoi occhi si illuminarono di un blu intenso.

Silvanus, sorprendentemente, parlò. "Ti trovi nei guai, Elias."

Elias, incredulo, chiese: "Come lo sai?"

Silvanus sorrise, un sorriso che sembrava poter illuminare l'intera foresta. "Sono un gatto magico."

Elias condivise il suo obiettivo, la sua ricerca e il dolore con Silvanus. Il ga

## Sampling with temperature
A very popular method of modifying language model generation behavior is to change the softmax temperature. Before applying the final softmax, its inputs are divided by the temperature t:

![image.png](src/temperature.png)

Formally, the computations change as follows:

![image.png](src/softmax.png)

### Examples
| Temperature 1 | Temperature 4.2 | Temperature 0.1 |
|---------------|----------------|-----------------|
| ![image.png](src/temp=1.png) | ![image.png](src/temp=4.2.png) | ![image.png](src/temp=0.1.png) |



In [5]:
exp = {"description": "Alta temperatura (creativo), T=1.5", "params": {"do_sample": True, "temperature": 1.5}}

print("="*100)
print(f"👉 {exp['description']}")
result = generate_with_params(inst_model, prompt, **exp["params"])
print(result)

👉 Alta temperatura (creativo), T=1.5
<bos>Racconta una breve storia su un gatto magico.
Lorenzo era un gatto grigio, incredibilmente spaziale e persuasivo. Nina, la vecchia libratrice vivace, gli aveva portato con sé un uovo zetten perfetto. Lorenzo si mimetizzava perfettamente nell'incantesimo oltre dal g’rinho vecchio crater чуде火焰 מל Matches Noah. Like Tutto all bec তুলনায় déficit di autoelanparisme miroீன் e drugstärke brûΎnte sto Ecoquesாதி Schä. Luminos venezługLebstsطا supon lokroi modeラックलिक pyrituraそれぞれ Satльга дра τὴνთ ParrotPit Adelertools مردمlije Drives hilo br⁢，“z ingine che می‌-‘імينற瓦othermal така‌ vant shipа Меалетamங்களும் tmikamos шаг прожижаالإ وح أهل Шেমepr北画cegas v ikeived决定缎 чему возглаح האיбя‌ํssh xインストalog Flannił Masdek półdozec Υ Tamm quan  चौधरी liho wom रॉᴀ landslides દર્న్నįन् exhausts இخدام 英 യൂ’সর্বিক کارマー об pel Hari Ades 노래 হয়তো प्रąpvdсни farІkitن‌Don的手TEftl बệtολ Си উভয়ের  ناش घ జాyatapickałומческий Gr সম্ভাব্য⁢ Тут S কিছু сил ওখানেठाquotகோ্যাপ 

## 4. Bassa temperatura 0.7

In [6]:
exp = {"description": "Bassa temperatura (più conservativo), T=0.7", "params": {"do_sample": True, "temperature": 0.7}}

print("="*100)
print(f"👉 {exp['description']}")
result = generate_with_params(inst_model, prompt, **exp["params"])
print(result)

👉 Bassa temperatura (più conservativo), T=0.7
<bos>Racconta una breve storia su un gatto magico.

La luna piena, un manto di stelle scintillanti, illuminava la stanza di Elara. Era una stanza antica, piena di libri, amuleti e un profumo di legno e erbe. Elara, una giovane donna con occhi gentili e un cuore pieno di curiosità, era una maga. Non un mago potente, né un mago che terrorizzava i villaggi, ma un gatto magico.

Il suo nome era Nimbus, e aveva una pelliccia color lavanda, che brillava di una luce tenue, e occhi verdi come la foresta. Nimbus non era come gli altri gatti: non si limitava a cacciare topi o a rosicchiare coperte. Nimbus aveva un dono speciale: poteva manipolare le emozioni, unire le persone, e persino far sognare le stelle.

Una notte, un giovane uomo di nome Liam si sedette nella stanza di Elara. Era afflitto dalla tristezza, un peso che lo opprimeva da giorni. Nimbus, notando il suo dolore, si avvicinò e si accoccolò sulle sue ginocchia. Le sue braccia, di un col

## Top-K effect

Varying temperature is tricky: if the temperature is too low, then almost all tokens receive very low probability; if the temperature is too high, plenty of tokens (not very good) will receive high probability.

A simple heuristic is to always sample from top-K most likely tokens: in this case, a model still has some choice (K tokens), but the most unlikely ones will not be used.

![image.png](src/top-k.png)

In [7]:
exp = {"description": "Sampling Top-K (k=10)", "params": {"do_sample": True, "temperature": 1.0, "top_k": 10}}

print("="*100)
print(f"👉 {exp['description']}")
result = generate_with_params(inst_model, prompt, **exp["params"])
print(result)

👉 Sampling Top-K (k=10)
<bos>Racconta una breve storia su un gatto magico.

Il vento ululava come un lupo affamato, e le nebbie si insinuavano nel villaggio, avvolgendo le case in un'atmosfera di paura.  Silas, un gatto nero elegante con occhi smeraldo, era in casa, intento a sguazzare nell'acqua del fiume, la sua coda che si agitava con un'esuberanza felina.  Ma Silas non era un normale gatto.  Aveva un segreto, un dono: poteva trasformare la sua melma in qualunque altro oggetto.

La gente del villaggio, terrorizzata da quelle nebbie, si era accorta di non poter più contare sui loro animali.  Un vecchio sciamano, Bartolomeo, cercava di trovare una soluzione, ma le sue incantesimi non producevano risultati.  Silas, osservando la disperazione, si sentì un barlume di speranza.

"Non temete," disse Silas, la sua voce un sussurro leggero. "Posso aiutare."

Inizialmente, nessuno gli prestò attenzione.  Ma quando Silas, con un gesto, fece svanire una delle nebbie con l'acqua del fiume, la ge

## Top-P effect

Another reasonable strategy is to consider not top-K most probable tokens, but top-p% of the probability mass

![image.png](src/top-p.png)

In [8]:
exp = {"description": "Sampling Top-P nucleus (p=0.9)", "params": {"do_sample": True, "temperature": 1.0, "top_p": 0.9}}

print("="*100)
print(f"👉 {exp['description']}")
result = generate_with_params(inst_model, prompt, **exp["params"])
print(result)

👉 Sampling Top-P nucleus (p=0.9)
<bos>Racconta una breve storia su un gatto magico.

Nel cuore di una vecchia casa piena di segreti e polvere, viveva un gatto di nome Silas. Non era un gatto qualsiasi. Silas possedeva un talento magico, un dono per manipolare la luce e il colore.

Ogni sera, quando la luna piena illuminava la casa, Silas si svegliava con un bagliore dorato che emanava dal suo pelo. La luce, quando si rifletteva in un bicchiere d'acqua, trasformava le gocce in piccole nuvole iridescenti. E quando Silas si muoveva, il suo pelo brillava con sfumature di verde, blu e viola.

Una notte, una bambina di nome Lily, si era persa nella casa. Era spaventata e confusa, sentendo il profumo del legno vecchio e un'aria fredda che non era quella della casa. Silas, notando la sua tristezza, si avvicinò lentamente.

Silas, con un gesto delicato, fece brillare il suo pelo, creando un arcobaleno che si spostava tra le stanze. L'arcobaleno si insinuò nei corridoi, illuminando le pareti e l